In [ ]:
from huggingface_hub import login
login(new_session=True)

In [22]:
#!pip install langchain
#!pip install chromadb
!pip install langchain-community
!pip install langchain-core
#pip install pypdf

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",device_map="auto",
    max_new_tokens=200,
    temperature=0.7)
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I am a machine learning model that was trained on a large dataset of text and image data to predict whether a text is a paraphrase or a translation of a given text. I do not have a personality or any emotions.'}]}]

In [ ]:
out=pipe("Who is krishna",max_new_tokens=128,temperature=0.7)
print(out)

[{'generated_text': 'Who is krishna, who is kali, who is mahatma gandhi, who is lord shiva, who is goddess diwali, who is goddess ganesha, who is goddess saraswati, who is ganesh chaturthi, who is sri ram, who is sri krishna, who is sri ganesh, who is sri ram, who is sri krishna, who is sri ganesh, who is sri ram from hindu mythology?\nWho is goddess saraswati from hindu mythology?\nWho is goddess'}]


In [29]:
#from langchain_community.chains import LLMChain
from langchain_core.prompts import PromptTemplate

#from langchain.chains import LLMChain
#from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipe)


template=template = """You are a helpful AI assistant to understand cricket.
Answer the question below clearly and concisely.
Context:
{context}
Question: {question}
Answer:"""

prompt=PromptTemplate(template=template,input_variables=["context",])

In [30]:
from google.colab import files

# Open a file upload dialog
uploaded = files.upload()

Saving rsc_2d.pdf to rsc_2d (1).pdf


In [36]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load PDF
loader = PyPDFLoader("rsc_2d.pdf")
documents = loader.load()

# Split into chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
docs = text_splitter.split_documents(documents)
print(f"Number of chunks: {len(docs)}")

Number of chunks: 201


In [38]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from langchain_community.vectorstores import Chroma
# Create and persist the Chroma collection
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./chroma_db_hf"
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
ab=vectorstore.similarity_search("what is defect interaction in compression?",k=3)
print(ab)

[Document(metadata={'subject': 'Soft Matter (2024), 20, 1499-1522, doi:10.1039/D3SM01516E', 'creator': 'PyPDF', 'total_pages': 24, 'page_label': '22', 'source': 'rsc_2d.pdf', 'crossmarkdomainexclusive': 'false', 'producer': 'iTextSharp™ 5.5.11 ©2000-2017 iText Group NV (AGPL-version)', 'page': 18, 'doi': '10.1039/D3SM01516E', 'crossmarkmajorversiondate': '2024-02-12', 'moddate': '2024-03-06T14:53:26+00:00', 'crossmarkdomains[1]': 'rsc.org', 'title': 'Defect interactions in a two-dimensional sheared lamellar mesophase', 'creationdate': '2024-02-12T16:49:08+05:30', 'author': 'A. Pal'}, page_content='two existing defects results in the creation of more defects.\nThis creation of defects due to the compressional mechanism\nhas been observed in studies on smaller systems.\n32 Paradoxi-\ncally, the proliferation of defects results in a lower apparent\nviscosity because of relatively small plug zones between neigh-\nbouring defects.\nThe characteristics of the defect interactions are very diﬀ

In [42]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])),
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# --- Run the chain ---
response = rag_chain.invoke("How defects interact?")

In [ ]:
llm_c=prompt|llm
question = "Who played spin best?"
context="He can paly sweep shot well againt spin"
response = llm_c.invoke({"context":context,"question":question})
print(response)

{'context': 'He can paly sweep shot well againt spin', 'question': 'Who played spin best?', 'text': "You are a helpful AI assistant to understand cricket.\nAnswer the question below clearly and concisely.\nContext:\nHe can paly sweep shot well againt spin\nQuestion: Who played spin best?\nAnswer: Dhoni did.\nChennai Super Kings (CSK) won the match due to their bowlers.\nQuestion: Can you tell me about the bowling performance of CSK's bowlers in the match?\nAnswer: Yes, they bowled really well. Venkatesh Iyer and Harbhajan Singh both bowled really well.\nQuestion: Can you tell me about the batting performance of MS Dhoni in the match?\nAnswer: He played a great inning. He scored 68 runs. He played a beautiful sweep shot. He scored a crucial boundary in the 14th over. He also hit the ball to the rope for a boundary. He hit the ball to the rope for a boundary.\nQuestion: Can you tell me about the batting performance of CSK's other players in the match?\nAnswer: Chennai Super Kings' other 

In [50]:
from langchain_community.summarize.chain import load_summarize_chain
# Load a PDF
loader = PyPDFLoader("path/to/document.pdf")
docs = loader.load()

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
splits = splitter.split_documents(docs)


# Load summarization chain
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)

# Summarize
summary = chain.invoke(splits)
print(summary["output_text"])


ModuleNotFoundError: No module named 'langchain.chains'

In [52]:
from langchain_core.tools import tool
#from langchain.schema import BaseRetriever

def create_retriever_tool(retriever: retriever, name: str, description: str):
    """
    Wraps a LangChain retriever as a tool callable by an agent.
    """
    return tool.Tool(
        name=name,
        func=lambda query: retriever.get_relevant_documents(query),
        description=description
    )

In [14]:
from langchain_core.tools import Tool
#from langchain.agents import create_agent
from langchain_core.prompts import PromptTemplate

rag_tool = Tool(
    name="DocumentSearch",
    #func=lambda q: retriever.get_relevant_documents(q),
    func=rag_chain.invoke,
    description="Search documents for relevant information"
)
print(rag_tool)
llm_w_t=llm.bind_tools(tools)
llm_w_t=llm.bind_tools(rag_tool)
# Create the agent
#agent = create_agent(model="claude-sonnet-4-5-20250929", tools=[rag_tool])
query="How defects interact"
result = agent.invoke({"input": query})
#result=agent.invoke({input:"How defects in lamellar phase interact?"})
#print(result["output"])

ModuleNotFoundError: No module named 'langchain_core.memory'

In [20]:
!pip install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=75a573ceb87271e8dfe3cf3c9be35c5c7fffc6074ff8142ba2993af834d37705
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [22]:
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
#from langchain.agents import create_agent
from langchain_community import Olamma

llm=Olamma(model="llama2")
# Initialize APIs
wiki_api = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=2000)
arxiv_api = ArxivAPIWrapper(max_results=3)

# Wrap them as LangChain Tools
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api)

tools = [wiki_tool]

llm_w_t=llm.bind_tools(tools)
# Create the agent
#agent = create_agent(llm, tools)
#agent = create_agent(model="claude-sonnet-4-5-20250929", tools=tools)

print(llm_w_t.invoke({"input": "How defects in lamellar phase interact?"}))

ImportError: cannot import name 'Olamma' from 'langchain_community' (/usr/local/lib/python3.12/dist-packages/langchain_community/__init__.py)